In [ ]:
import random
from PIL import Image
from ipywidgets import Button, VBox, Output, HBox
from IPython.display import display, clear_output
import time
import os

class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def __str__(self):
        return f"{self.rank}_of_{self.suit}.png"

class Deck:
    def __init__(self):
        ranks = [str(num) for num in range(2, 11)] + ['Jack', 'Queen', 'King', 'Ace']
        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        self.cards = [Card(rank, suit) for rank in ranks for suit in suits]  # Erzeugt alle Karten im Deck
        random.shuffle(self.cards)  # Mischt das Deck

    def draw_card(self):
        if len(self.cards) == 0:  # Überprüft, ob das Deck leer ist
            return None
        return self.cards.pop()  # Zieht eine Karte aus dem Deck

class Blackjack:
    def __init__(self):
        self.deck = Deck()  # Erstellt ein Deck für das Spiel
        self.player_hand = []  # Initialisiert die Hand des Spielers
        self.dealer_hand = []  # Initialisiert die Hand des Dealers
        self.output = Output()  # Erstellt eine Output-Widget für die Anzeige
        self.btn_hit = Button(description="Hit")  # Erstellt den "Hit" Button
        self.btn_stand = Button(description="Stand")  # Erstellt den "Stand" Button
        self.btn_restart = Button(description="Restart")  # Erstellt den "Restart" Button

        # Verknüpft die Button-Events mit den entsprechenden Methoden
        self.btn_hit.on_click(self.hit)
        self.btn_stand.on_click(self.stand)
        self.btn_restart.on_click(self.restart)

        # Erstellt eine Button-Box, um die Buttons anzuordnen
        self.buttons_box = HBox([self.btn_hit, self.btn_stand])

    def deal_initial(self):
        # Austeilen der Anfangshand für Spieler und Dealer
        self.player_hand.append(self.deck.draw_card())  # Spieler zieht eine Karte
        self.dealer_hand.append(self.deck.draw_card())  # Dealer zieht eine Karte
        self.player_hand.append(self.deck.draw_card())  # Spieler zieht eine zweite Karte
        self.dealer_hand.append(self.deck.draw_card())  # Dealer zieht eine zweite Karte
        # Die erste Karte des Dealers wird verdeckt
        self.dealer_hand[0].hidden = True

    def display_table(self, dealer_score=False, blackjack=False):
        # Anzeigen des Spieltisches mit den aktuellen Händen
        with self.output:
            clear_output(wait=True)  # Löscht den vorherigen Output
            table_img = load_image("Player_table.png")  # Lädt das Hintergrundbild des Spieltisches
            table_img = table_img.resize((800, 555))  # Ändert die Größe des Bildes
            card_width = 73
            card_height = 98
            padding = 5

            start_x_player = 200
            start_y_player = 350
            player_score = self.calculate_score(self.player_hand)  # Berechnet den Punktestand des Spielers
            # Anzeigen der Karten des Spielers
            for i, card in enumerate(self.player_hand):
                card_img = load_image(card.__str__())
                card_img = card_img.resize((int(card_width * 0.8), int(card_height * 0.8)))
                table_img.paste(card_img, (start_x_player + i * (card_width + padding), start_y_player))

            start_x_dealer = 200
            start_y_dealer = 100
            # Anzeigen der Karten des Dealers
            if dealer_score:
                dealer_score = self.calculate_score(self.dealer_hand)  # Berechnet den Punktestand des Dealers
            for i, card in enumerate(self.dealer_hand):
                card_img = load_image(card.__str__())
                card_img = card_img.resize((int(card_width * 0.7), int(card_height * 0.7)))
                if i == 0:  # Die erste Karte des Dealers wird verdeckt angezeigt
                    if card.hidden:
                        card_img = load_image("backcard.png")
                        card_img = card_img.resize((int(card_width * 0.7), int(card_height * 0.7)))
                table_img.paste(card_img, (start_x_dealer + i * (card_width + padding), start_y_dealer))

            display(table_img)  # Anzeigen des Spieltisches
            if blackjack:
                print("Blackjack!")  # Meldung, wenn ein Spieler Blackjack hat
            else:
                print("Player's Score:", player_score)  # Anzeigen des Punktestands des Spielers
                if dealer_score and len(self.dealer_hand) == 2:  # Wenn die verdeckte Karte des Dealers gezogen wurde
                    print("Dealer's Score:", dealer_score)
                elif dealer_score:
                    print("Dealer's Score:", dealer_score)

    def hit(self, button):
        # Methode zum Ziehen einer Karte durch den Spieler
        self.player_hand.append(self.deck.draw_card())  # Spieler zieht eine Karte
        self.display_table()  # Aktualisierung des Spieltisches
        player_score = self.calculate_score(self.player_hand)  # Berechnet den Punktestand des Spielers
        if player_score == 21:
            self.display_table(blackjack=True)  # Wenn der Spieler Blackjack hat
            self.stand(None)
        elif player_score > 21:
            self.dealer_hand[0].hidden = False  # Die verdeckte Karte des Dealers wird gezeigt
            self.display_table(dealer_score=True)  # Anzeigen des Dealer-Scores
            self.determine_winner()  # Bestimmen des Gewinners
            self.buttons_box.children = [self.btn_restart]

    def stand(self, button):
        # Methode zum Beenden des Spielers und zum Ziehen von Karten durch den Dealer
        self.btn_hit.disabled = True
        self.btn_stand.disabled = True
        self.dealer_hand[0].hidden = False  # Die verdeckte Karte des Dealers wird gezeigt
        time.sleep(0.1)

        while self.calculate_score(self.dealer_hand) < 17:  # Dealer zieht, bis sein Punktestand 17 oder höher ist
            self.dealer_hand.append(self.deck.draw_card())
            time.sleep(0.1)
        
        self.display_table(dealer_score=True)  # Aktualisierung des Spieltisches nach Abschluss des Zugs des Dealers
        self.determine_winner()  # Bestimmen des Gewinners
        self.buttons_box.children = [self.btn_restart]

    def get_card_value(self, card):
        # Methode zur Bestimmung des Werts einer Karte
        if card.rank in ['Jack', 'Queen', 'King']:
            return 10
        elif card.rank == 'Ace':
            return 11
        else:
            return int(card.rank)

    def calculate_score(self, hand):
        # Methode zur Berechnung des Gesamtpunktestands einer Hand
        score = 0
        num_aces = 0
        for card in hand:
            if card.rank == 'Ace':
                num_aces += 1
            score += self.get_card_value(card)
        
        while score > 21 and num_aces:
            score -= 10
            num_aces -= 1
        
        return score

    def determine_winner(self):
        # Methode zur Bestimmung des Gewinners des Spiels
        player_score = self.calculate_score(self.player_hand)
        dealer_score = self.calculate_score(self.dealer_hand)
        
        if player_score > 21:
            print("Bust! Spieler verliert.")
        elif dealer_score > 21 or player_score > dealer_score:
            print("Player gewinnt!")
        elif dealer_score > player_score:
            print("Dealer gewinnt!")
        else:
            print("Unentschieden!")

    def restart(self, button):
        # Methode zum Neustarten des Spiels
        self.player_hand = []
        self.dealer_hand = []
        self.deck = Deck()  # Neues Deck erstellen
        self.btn_hit.disabled = False
        self.btn_stand.disabled = False
        self.deal_initial()  # Austeilen der Anfangshand
        self.display_table()  # Anzeigen des Spieltisches
        self.buttons_box.children = [self.btn_hit, self.btn_stand]

    def play(self):
        # Methode zum Starten des Spiels
        self.deal_initial()  # Austeilen der Anfangshand
        player_score = self.calculate_score(self.player_hand)
        if player_score == 21:  # Wenn der Spieler Blackjack hat
            self.display_table(blackjack=True)
            self.stand(None)
        else:
            self.display_table()  # Anzeigen des Spieltisches
            display(VBox([self.output, self.buttons_box]))  # Anzeigen des Outputs und der Buttons

def load_image(img):
    path = os.path.join(img_folder, img)         
    return Image.open(path)   
            
img_folder = 'card_imgs'
game = Blackjack()  # Instanzierung des Spiels
game.play()  # Starten des Spiels
